# Tema 12: NVidia CUDA avanzado

## A - Warps

In [1]:
### EVITAR ERRORES

!uv pip install -q --system numba-cuda==0.4.0

from numba import config
config.CUDA_ENABLE_PYNVJITLINK = 1

In [ ]:
import numpy as np
from numba import cuda

threads_per_block = 1024
blocks = 1024
n = blocks*threads_per_block

h_a = np.random.rand(n).astype(np.float32)
d_a = cuda.to_device(h_a)

@cuda.jit
def experiment(a, convergence):
    i = cuda.grid(1)
    block_idx = cuda.blockIdx.x
    if convergence == True:
        if (block_idx%4==0):
            a[i] = a[i]+2
        elif (block_idx%4==1):
            a[i] = np.sin(a[i])
        elif (block_idx%4==2):
            a[i] = np.cos(a[i])
        elif (block_idx%4==3):
            a[i] = a[i]**0.5
    else:
        if (i%4==0):
            a[i] = a[i]+2
        elif (i%4==1):
            a[i] = np.sin(a[i])
        elif (i%4==2):
            a[i] = np.cos(a[i])
        elif (i%4==3):
            a[i] = a[i]**0.5

In [ ]:
%timeit experiment[blocks, threads_per_block](d_a, True); cuda.synchronize()
%timeit experiment[blocks, threads_per_block](d_a, False); cuda.synchronize()

730 µs ± 47.1 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.04 ms ± 199 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## B - Acceso coalescente a memoria global

In [ ]:
import numpy as np
from numba import cuda

n = 1024*1024 # 10M
threads_per_block = 1024
blocks = 1024

# Input Vectors of length 10M
h_a = np.ones(n).astype(np.float32)
h_b = h_a.copy().astype(np.float32)

# Output Vector
d_a = cuda.to_device(h_a)
d_b = cuda.to_device(h_b)
d_out = cuda.device_array_like(d_a)

@cuda.jit
def add_experiment(a, b, out, coalesced):
    if coalesced == True:
        i = cuda.grid(1)
        out[i] = a[i] + b[i]
    else:
        thread_idx = cuda.threadIdx.x
        block_idx = cuda.blockIdx.x
        out[thread_idx*threads_per_block+block_idx] = \
            a[thread_idx*threads_per_block+block_idx] + \
            b[thread_idx*threads_per_block+block_idx]

In [ ]:
%timeit add_experiment[blocks, threads_per_block](d_a, d_b, d_out, True); cuda.synchronize()
%timeit add_experiment[blocks, threads_per_block](d_a, d_b, d_out, False); cuda.synchronize()

__Ejercicio: sumar filas y columnas de matriz__

In [ ]:
import numpy as np
from numba import cuda

@cuda.jit
def row_sums(a, sums, n):
    idx = cuda.grid(1)
    sum = 0.0
    for i in range(n):
        sum += a[idx][i]
    sums[idx] = sum

@cuda.jit
def col_sums(a, sums, n):
    idx = cuda.grid(1)
    sum = 0.0
    for i in range(n):
        sum += a[i][idx]
    sums[idx] = sum

n = 8192
threads_per_block = 128
blocks = n // threads_per_block  # 8192 / 128 = 64

# Matriz de entrada
h_a = np.ones((n, n), dtype=np.float32)

# Copia a la GPU
d_a = cuda.to_device(h_a)
d_rows = cuda.device_array(n, dtype=np.float32)
d_cols = cuda.device_array(n, dtype=np.float32)

# Lanzamiento de kernels
row_sums[blocks, threads_per_block](d_a, d_rows, n)
col_sums[blocks, threads_per_block](d_a, d_cols, n)

# Copiar resultados a CPU
h_rows = d_rows.copy_to_host()
h_cols = d_cols.copy_to_host()

# Comprobación de resultados
np.testing.assert_equal(h_rows, h_a.sum(axis=1))
np.testing.assert_equal(h_cols, h_a.sum(axis=0))

# Benchmark
print("Tiempo suma de filas:")
%timeit row_sums[blocks, threads_per_block](d_a, d_rows, n); cuda.synchronize()
print("Tiempo suma de columnas:")
%timeit col_sums[blocks, threads_per_block](d_a, d_cols, n); cuda.synchronize()


## C - Kernels bidimensionales y tridimensionales

In [ ]:
import numpy as np
from numba import cuda

@cuda.jit
def get_2D_indices(A):
    x, y = cuda.grid(2) # Obtenemos las dos dimensiones
    # Equivalente a:
    # x = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    # y = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y

    # Escribimos índice x + '.' + índice y
    A[x][y] = x + y / 10

d_A = cuda.device_array(shape=(4,4), dtype=np.float32)
    # Matriz 4x4 en la GPU

blocks = (2, 2) # Grid = 2x2 bloques
threads_per_block = (2, 2) # Bloque = 2x2 threads

np.set_printoptions(precision=1, floatmode="fixed")
get_2D_indices[blocks, threads_per_block](d_A)

print(d_A.copy_to_host())

__Ejercicio: sumar matrices con kernel 2D__

In [ ]:
import numpy as np
from numba import cuda

n = 4096

# TODO: completar
@cuda.jit
def matrix_add(a, b, out, coalesced):
    x, y = cuda.grid(2)
    if coalesced == True:
      out[y][x] = a[y][x] + b[y][x]
    else:
      out[x][y] = a[x][y] + b[x][y]

threads_per_block = (32, 32)  # 2D block
blocks = (128, 128) # 2D grid

h_a = np.arange(n*n).reshape(n,n).astype(np.float32)
h_b = h_a.copy().astype(np.float32)

# TODO: copia a la GPU y reserva para la salida

d_a = cuda.to_device(h_a)
d_b = cuda.to_device(h_b)
d_out = cuda.device_array(shape=(n,n), dtype=np.float32)

matrix_add[blocks, threads_per_block](d_a, d_b, d_out, True)
h_out = d_out.copy_to_host()

# TODO: invocación kernel y obtención resultados
truth = h_a+h_b
np.testing.assert_equal(h_out, truth)
print("Coalescente")
%timeit matrix_add[blocks, threads_per_block](d_a, d_b, d_out, True); cuda.synchronize()
print("No coalescente")
%timeit matrix_add[blocks, threads_per_block](d_a, d_b, d_out, False); cuda.synchronize()

## D - Memoria compartida

In [ ]:
import numpy as np
from numba import types, cuda

n=5

@cuda.jit
def swap_with_shared(vector, swapped):
    temp = cuda.shared.array(n, dtype=types.int32)
    idx = cuda.grid(1)
    temp[idx] = vector[idx]

    cuda.syncthreads()

    swapped[idx] = temp[n-1 - cuda.threadIdx.x]
		# swap elements

h_vector = np.arange(n).astype(np.int32)
print("Antes:",h_vector)
h_swapped = np.zeros_like(h_vector)

d_vector = cuda.to_device(h_vector)
d_swapped = cuda.device_array(shape=(n,), dtype=np.int32)

swap_with_shared[1, n](d_vector, d_swapped)
result = d_swapped.copy_to_host()
print("Después:",result)

__Ejercicio: trasponer matriz con coalescencia usando memoria compartida__

In [5]:
from numba import cuda, types as numba_types
import numpy as np

n = 1024 * 1024  # 1M

@cuda.jit
def transpose(a, transposed):
    x, y = cuda.grid(2)
    if x < transposed.shape[0] and y < transposed.shape[1]:
        transposed[x][y] = a[y][x]

@cuda.jit
def tile_transpose(a, transposed):
    TILE_DIM = 32
    tile = cuda.shared.array((TILE_DIM, TILE_DIM), numba_types.float32)

    a_row, a_col = cuda.grid(2)

    row = cuda.threadIdx.y
    col = cuda.threadIdx.x

    if a_row < a.shape[0] and a_col < a.shape[1]:
        tile[row, col] = a[a_row, a_col]

    cuda.syncthreads()

    t_row = cuda.blockIdx.x * TILE_DIM + row
    t_col = cuda.blockIdx.y * TILE_DIM + col

    if t_col < transposed.shape[0] and t_row < transposed.shape[1]:
        transposed[t_col, t_row] = tile[col, row]

threads_per_block = (32, 32)  # 2D blocks
blocks = (1024 // 32, 1024 // 32)  # 2D grid = (32, 32)

# 1024x1024 input and output matrices
h_a = np.arange(n).reshape((1024, 1024)).astype(np.float32)
d_a = cuda.to_device(h_a)
d_transposed = cuda.device_array(shape=(1024, 1024), dtype=np.float32)

# Invocación a traspose y comprobación
transpose[blocks, threads_per_block](d_a, d_transposed)
result = d_transposed.copy_to_host()
expected = h_a.T
np.testing.assert_equal(result, expected)

# Invocación a tile_traspose y comprobación
tile_transpose[blocks, threads_per_block](d_a, d_transposed)
result = d_transposed.copy_to_host()
expected = h_a.T
np.testing.assert_equal(result, expected)

%timeit transpose[blocks, threads_per_block](d_a, d_transposed); cuda.synchronize()
%timeit tile_transpose[blocks, threads_per_block](d_a, d_transposed); cuda.synchronize()

139 µs ± 8.63 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
191 µs ± 7.84 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## F - Comparación accesos a memoria

In [ ]:
from numba import jit, cuda, types as numba_types
import numpy as np

n = 1024*1024 # 1M

# ----------------------------

h_a = np.arange(n).reshape((1024,1024)).astype(np.float32)
h_out = np.zeros(n).reshape((1024,1024)).astype(np.float32)

@jit
def transpose_CPU(a, transposed):
    for i in range(1024):
        for j in range(1024):
            transposed[i,j] = a[j,i]

transpose_CPU(h_a, h_out)
expected = h_a.T
np.testing.assert_equal(h_out, expected)

# ----------------------------

h_a = np.arange(n).reshape((1024,1024)).astype(np.float32)
h_out = np.zeros(n).reshape((1024,1024)).astype(np.float32)
d_a = cuda.to_device(h_a)
d_out = cuda.device_array(shape=(1024,1024), dtype=np.float32)

@cuda.jit
def transpose1thread(a, transposed):
    for i in range(1024):
        for j in range(1024):
            transposed[i,j] = a[j,i]

transpose1thread[1, 1](d_a, d_out)
expected = h_a.T
h_out=d_out.copy_to_host()
np.testing.assert_equal(h_out, expected)

# ----------------------------

h_a = np.arange(n).reshape((1024,1024)).astype(np.float32)
h_out = np.zeros(n).reshape((1024,1024)).astype(np.float32)
d_a = cuda.to_device(h_a)
d_out = cuda.device_array(shape=(1024,1024), dtype=np.float32)

@cuda.jit
def transpose1024thread(a, transposed):
    i = cuda.threadIdx.x
    for j in range(1024):
        transposed[i,j] = a[j,i]

transpose1024thread[1, 1024](d_a, d_out)
expected = h_a.T
h_out=d_out.copy_to_host()
np.testing.assert_equal(h_out, expected)

# ----------------------------

h_a = np.arange(n).reshape((1024,1024)).astype(np.float32)
h_out = np.zeros(n).reshape((1024,1024)).astype(np.float32)
d_a = cuda.to_device(h_a)
d_out = cuda.device_array(shape=(1024,1024), dtype=np.float32)

@cuda.jit
def transpose1Mthreads(a, transposed):
    (i,j) = cuda.grid(2)
    transposed[i,j] = a[j,i]

blocks=(32,32)
threads_per_block=(32,32)
transpose1Mthreads[blocks, threads_per_block](d_a, d_out)
h_out=d_out.copy_to_host()
expected = h_a.T
np.testing.assert_equal(h_out, expected)

# ----------------------------

h_a = np.arange(n).reshape((1024,1024)).astype(np.float32)
h_out = np.zeros(n).reshape((1024,1024)).astype(np.float32)
d_a = cuda.to_device(h_a)
d_out = cuda.device_array(shape=(1024,1024), dtype=np.float32)

nthreads=32
nblocks=int(1024/nthreads)
@cuda.jit
def tile_transpose(a, transposed):
    tile = cuda.shared.array((nthreads, nthreads), numba_types.float32)
    a_row, a_col = cuda.grid(2)
    tile[cuda.threadIdx.x, cuda.threadIdx.y] = a[a_row, a_col]
    cuda.syncthreads()
    transposed[a_col, a_row] = tile[cuda.threadIdx.x, cuda.threadIdx.y]

blocks=(nblocks,nblocks)
threads_per_block=(nthreads, nthreads)
tile_transpose[blocks, threads_per_block](d_a, d_out)
h_out=d_out.copy_to_host()
expected = h_a.T
np.testing.assert_equal(h_out, expected)


# ----------------------------

h_a = np.arange(n).reshape((1024,1024)).astype(np.float32)
h_out = np.zeros(n).reshape((1024,1024)).astype(np.float32)
d_a = cuda.to_device(h_a)
d_out = cuda.device_array(shape=(1024,1024), dtype=np.float32)

nthreads=32
nblocks=int(1024/nthreads)
ncols=33
@cuda.jit
def tile_transpose2(a, transposed):
    tile = cuda.shared.array((nthreads, ncols), numba_types.float32)
    a_row, a_col = cuda.grid(2)
    tile[cuda.threadIdx.x, cuda.threadIdx.y] = a[a_row, a_col]
    cuda.syncthreads()
    transposed[a_col, a_row] = tile[cuda.threadIdx.x, cuda.threadIdx.y]

blocks=(nblocks,nblocks)
threads_per_block=(nthreads, nthreads)
tile_transpose[blocks, threads_per_block](d_a, d_out)
h_out=d_out.copy_to_host()
expected = h_a.T
np.testing.assert_equal(h_out, expected)

In [ ]:
%timeit transpose_CPU(h_a, h_out)

In [ ]:
%timeit transpose1thread[1, 1](d_a, d_out); cuda.synchronize()

In [ ]:
%timeit transpose1024thread[1, 1024](d_a, d_out); cuda.synchronize()

In [ ]:
%timeit transpose1Mthreads[(32,32), (32,32)](d_a, d_out); cuda.synchronize()

In [ ]:
%timeit tile_transpose[(32,32), (32,32)](d_a, d_out); cuda.synchronize()
%timeit tile_transpose[(64,64), (16,16)](d_a, d_out); cuda.synchronize()

In [ ]:
%timeit tile_transpose2[(32,32), (32,32)](d_a, d_out); cuda.synchronize()